ARGAN : Attentive Recurrent Generative Adversarial Network
for Shadow Detection and Removal

# import lib & data

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# import lib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import os

import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from skimage import color

# mount to drive
from google.colab import drive
drive.mount('/content/drive')

# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# load images from Google Drive
#!ls '/content/drive/My Drive/KU/4/ISTD_Dataset/train/'
img_path = '/content/drive/My Drive/KU/4/ISTD_Dataset/train/'
test_path = '/content/drive/My Drive/KU/4/ISTD_Dataset/test/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cpu


Functions

In [ ]:
# display image
def imshow(image):
  # numpy
  npimage = image.detach().numpy()
  plt.imshow(np.transpose(npimage, (1,2,0)))
  plt.show()

# Save output tensor as image
def save_batch(images, nrow, PATH):
  img = torchvision.utils.make_grid(images, nrow=nrow)
  img_out = np.transpose(img.detach().numpy().astype('float64'), (1,2,0))
  img_out = (255*img_out).astype('uint8')
  img_out = Image.fromarray(img_out)
  img_out.save(PATH)

def save_batch_LAB(images, nrow, PATH):
  img = torchvision.utils.make_grid(images, nrow=nrow)
  img_out = np.transpose(img.detach().numpy().astype('float64'), (1,2,0))
  img_out = color.lab2rgb(img_out)
  img_out = (255*img_out).astype('uint8')
  img_out = Image.fromarray(img_out)
  img_out.save(PATH)

Dataloader

In [ ]:
# My dataset loading function
def make_dataset(root, test) -> list:
  dataset = []
  # sub folder names of data set
  if test is True:
    src_dir = 'test_A'
    matt_dir = 'test_B'
    free_dir = 'test_C'
  else:
    src_dir = 'train_A'
    matt_dir = 'train_B'
    free_dir = 'train_C'

  # file names of dataset
  src_fnames = sorted(os.listdir(os.path.join(root, src_dir)))
  matt_fnames = sorted(os.listdir(os.path.join(root,matt_dir)))
  free_fnames = sorted(os.listdir(os.path.join(root,free_dir)))

  # matching datasets by name
  # same fname for triplets
  for src_fname in src_fnames:
    # source image (image with shadow)
    src_path = os.path.join(root,src_dir,src_fname)
    if  src_fname in matt_fnames:
      # shadow matte image
      matt_path = os.path.join(root,matt_dir,src_fname)
      if src_fname in free_fnames:
        # shadow free image
        free_path = os.path.join(root,free_dir,src_fname)
        # if triplets exists append to dataset
        temp = (src_path, matt_path, free_path)
        dataset.append(temp)
      # if one of triplets missing do NOT append to dataset
      else:
        print(free_fname, 'Shadow free file missing')
        continue
    else:
      print(matt_fname, 'Shadow matte file missing')
      continue

  return dataset


class ARGAN_Dataset(torchvision.datasets.vision.VisionDataset):
  # ARGAN dataset class composed of 3 func
  def __init__(self, root, loader=torchvision.datasets.folder.default_loader,
               is_test=False, src_trans=None, matt_trans=None):
    super().__init__(root, transform=src_trans, target_transform=matt_trans)
    self.test = is_test
    # Custom dataset loader for Training
    samples = make_dataset(self.root, test=is_test)
    self.loader = loader
    self.samples = samples
    self.trans2tensor = transforms.ToTensor()
    # train data list
#    self.src_samples = [s[0] for s in samples]
#    self.matt_samples = [s[1] for s in samples]
#    self.free_samples = [s[2] for s in samples]

  # Get single data
  def __getitem__(self, index):
    # load training data
    src_path, matt_path, free_path = self.samples[index]
    src_sample = self.loader(src_path)
    matt_sample = self.loader(matt_path)
    free_sample = self.loader(free_path)

    matt = self.trans2tensor(matt_sample)
    free = self.trans2tensor(free_sample)

    # transform data if required
    if self.transform is not None:
      # transform for RGB image : Shadow image and Shadow free image
      src_sample = self.transform(src_sample)
      free_sample = self.transform(free_sample)
    if self.target_transform is not None:
      # transform for Binary image : Shaode Matte
      matt_sample = self.target_transform(matt_sample)

    if self.test is False:
      return src_sample, matt_sample, free_sample
    else:
      return src_sample, matt_sample, free_sample, matt, free

  # Get dataset length
  def __len__(self):
    return len(self.samples)

Transforms

In [ ]:
# image Transforms
# image size 256x256 used for training _ from paper
img2tensor = transforms.Compose([
                                 transforms.Resize(size=(256,256)),
                                 transforms.ToTensor()
                                 # additional tasks
])
matt2tensor = transforms.Compose([
                                  transforms.Resize(size=(256,256)),
                                  transforms.Grayscale(1),
                                  transforms.ToTensor()
                                  # additional tasks
])

# Load images
batch_num = 4
dprow = 2

#train_img = ARGAN_Dataset(img_path, src_trans=img2tensor, matt_trans=matt2tensor, is_test=False)
#trainloader = torch.utils.data.DataLoader(train_img, batch_size=batch_num, shuffle=True)
"""
test_img = ARGAN_Dataset(test_path, src_trans=img2tensor, matt_trans=matt2tensor, is_test=True)
testloader = torch.utils.data.DataLoader(test_img, batch_size=4, shuffle=False)
"""

'\ntest_img = ARGAN_Dataset(test_path, src_trans=img2tensor, matt_trans=matt2tensor, is_test=True)\ntestloader = torch.utils.data.DataLoader(test_img, batch_size=4, shuffle=False)\n'

# Check Dataset

Loaded Dataset

In [ ]:
print(train_img)

#for i, (src,matt) in enumerate(trainloader):
dataiter = iter(trainloader)
print(type(dataiter))
images, mattes, frees = dataiter.next()

print(images.shape)
print(mattes.shape)
print(frees.shape)

imshow(torchvision.utils.make_grid(images, nrow=dprow))
imshow(torchvision.utils.make_grid(mattes, nrow=dprow))
imshow(torchvision.utils.make_grid(frees, nrow=dprow))

#imshow(frees)

Dataset ARGAN_Dataset
    Number of datapoints: 1330
    Root location: /content/drive/My Drive/KU/4/ISTD_Dataset/train/
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=bilinear)
               ToTensor()
           )
Target transform: Compose(
                      Resize(size=(256, 256), interpolation=bilinear)
                      Grayscale(num_output_channels=1)
                      ToTensor()
                  )
<class 'torch.utils.data.dataloader._SingleProcessDataLoaderIter'>


KeyboardInterrupt: ignored

Test Dataset

In [ ]:
print(test_img)

#for i, (src,matt) in enumerate(trainloader):
testiter = iter(testloader)

images, mattes, frees = testiter.next()

print(images.shape)
print(mattes.shape)
print(frees.shape)

imshow(torchvision.utils.make_grid(images, nrow=dprow))
imshow(torchvision.utils.make_grid(mattes, nrow=dprow))
imshow(torchvision.utils.make_grid(frees, nrow=dprow))

#imshow(frees)

# Generative Network

Convolutional LSTM

In [ ]:
# LSTM layer
class ConvLSTM(nn.Module):
    def __init__(self, inp_dim, hid_dim, out_dim=None):
        super(ConvLSTM, self).__init__()
        self.in_dim = inp_dim + hid_dim
        self.hidden_dim = hid_dim
        # Same output Channel as input Channel
        if out_dim is None:
            self.out_dim = inp_dim
        else:
            self.out_dim = out_dim
        self.conv_i = nn.Conv2d(self.in_dim, self.out_dim, 3, 1, 1)
        self.conv_f = nn.Conv2d(self.in_dim, self.out_dim, 3, 1, 1)
        self.conv_c = nn.Conv2d(self.in_dim, self.out_dim, 3, 1, 1)
        self.conv_o = nn.Conv2d(self.in_dim, self.out_dim, 3, 1, 1)
        self.sig = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x, c_prev, h_prev):
        # input X and hidden state H_prev
        xh = torch.cat((x, h_prev), 1)
        i = self.sig(self.conv_i(xh))
        f = self.sig(self.conv_f(xh))
        c = (f * c_prev) + (i * self.tanh(self.conv_c(xh)))
        o = self.sig(self.conv_o(xh))
        h = o * self.tanh(c)
        # C_next : cell output, H_next : hidden state
        return c, h

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return torch.zeros(batch_size, self.hidden_dim, height, width)

Generator

In [ ]:
# Conv + BN + LReLU
class ConvL(nn.Module):
    def __init__(self, inp_ch, out_ch, k=None, s=None, p=None):
        super(ConvL, self).__init__()
        # (3, 1, 1) default
        if k is None:
            k = 3
            s = 1
            p = 1
        # default params
        elif s is None:
            s = 1
        elif p is None:
            p = 0
        self.conv = nn.Sequential(
            nn.Conv2d(inp_ch, out_ch, kernel_size=k, stride=s, padding=p),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU())

    def forward(self, x):
        return self.conv(x)

# ConvT + BN + LReLU
class DConvL(nn.Module):
    def __init__(self, inp_ch, out_ch, k=None, s=None, p=None):
        super(DConvL, self).__init__()
        # (3, 1, 1) keep spatial resolution
        if k is None:
            k = 3
            s = 1
            p = 1
        # default params
        elif s is None:
            s = 1
        elif p is None:
            p = 0
        self.dconv = nn.Sequential(
            nn.ConvTranspose2d(inp_ch, out_ch,
                               kernel_size=k, stride=s, padding=p),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU())

    def forward(self, x):
        return self.dconv(x)

# Attention Detector : 10 (Conv + BN + LReLU) layers
class AttDet(nn.Module):
    def __init__(self):
        super(AttDet, self).__init__()
        self.block = nn.Sequential(
            ConvL(4, 8), ConvL(8, 8), ConvL(8, 16), ConvL(16, 16),
            ConvL(16, 16), ConvL(16, 32), ConvL(32,32),
            ConvL(32, 64), ConvL(64, 64), ConvL(64, 64)
        )

    def forward(self, x):
        out = self.block(x)
        return out

# Removal Encoder : 8 Conv + 8 DConv +3 Conv Layers
class REncoder(nn.Module):
    def __init__(self):
        super(REncoder, self).__init__()
        # CONV LAYERS : extract feature
        self.conv0 = ConvL(3, 64, 3, 2, 3)
        self.conv1 = ConvL(64, 128, 3, 2, 2)
        self.conv2 = ConvL(128, 256, 3, 2, 2)
        self.conv3 = ConvL(256, 512, 3, 2, 2)
        self.conv4 = ConvL(512, 512, 3, 2, 2)
        self.conv5 = ConvL(512, 512, 3, 2, 2)
        self.conv6 = ConvL(512, 512, 3, 2, 2)
        self.conv7 = ConvL(512, 512, 3, 2, 2)

        # DECONV LAYERS : generate image with feature data
        self.dconv0 = DConvL(512, 512, 4, 2, 2)
        self.dconv1 = DConvL(512, 512, 4, 2, 2)
        self.dconv2 = DConvL(512, 512, 4, 2, 2)
        self.dconv3 = DConvL(512, 512, 4, 2, 2)
        self.dconv4 = DConvL(512, 256, 4, 2, 2)
        self.dconv5 = DConvL(256, 128, 4, 2, 2)
        self.dconv6 = DConvL(128, 64, 4, 2, 2)
        self.dconv7 = DConvL(64, 3, 4, 2, 3)

        # Convert to Neg residual
        self.rem0 = ConvL(3, 3)
        self.rem1 = ConvL(3, 3)
        self.rem2 = nn.Sequential(nn.Conv2d(3,3, kernel_size=3,
                                            stride=1, padding=1),
                                  nn.Sigmoid())

    def forward(self, x, att_map):
        x0 = self.conv0(x)
        x1 = self.conv1(x0)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        x7 = self.conv7(x6)

        xx = self.dconv0(x7)
        xx += x6
        xx = self.dconv1(xx)
        xx += x5
        xx = self.dconv2(xx)
        xx += x4
        xx = self.dconv3(xx)
        xx += x3
        xx = self.dconv4(xx)
        xx += x2
        xx = self.dconv5(xx)
        xx += x1
        xx = self.dconv6(xx)
        xx += x0
        xx = self.dconv7(xx)

        xx = self.rem0(xx)
        xx = self.rem1(xx)
        xx = self.rem2(xx)

        res = xx * att_map
        out = res + x

        return out

In [ ]:
# Generative Network
class Gen(nn.Module):
    def __init__(self, batch_size=None, step_num=None):
        super(Gen, self).__init__()
        self.batch_size = batch_size
        self.step = step_num
        # Attention Detector
        self.attL = []
        self.remE = []
        for i in range(self.step):
          self.attL.append(AttDet())
          self.remE.append(REncoder())
        self.attL = nn.ModuleList(self.attL)
        self.remE = nn.ModuleList(self.remE)
        #        self.attL = AttDet()
        # Convolutional LSTM cell
#        self.lstm = ConvLSTM(inp_dim=64, hid_dim=64)
        # init hidden state
#        self.hidden = self.lstm.init_hidden(self.batch_size, (256,256))
        # Attention Map
        self.attM = nn.Sequential(
                  nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1),
                  nn.Sigmoid()
        )
        # Removal Encoder
#        self.remE = REncoder()

#    def init_h(self):
#      self.hidden = self.lstm.init_hidden(self.batch_size, (256,256))

    def forward(self, x):
      in_batch = x.shape[0]
      if in_batch != self.batch_size:
        self.batch_size = in_batch
#      self.hidden = self.lstm.init_hidden(self.batch_size, (256,256))
      with torch.autograd.set_detect_anomaly(True):
        # attention map & output tensor
        empt = torch.zeros(self.batch_size, 1, 256, 256).to(device)
        att_map = torch.empty(self.step, self.batch_size, 1, 256, 256).to(device)
        out = torch.empty(self.step , self.batch_size, 3, 256, 256).to(device)
#        lstm_out = torch.zeros(self.batch_size, 64, 256, 256).to(device)
#        self.hidden = self.hidden.to(device)

        # for N progressive steps
        for i in range(self.step):
            if (i == 0):
                xx = torch.cat([x, empt], dim=1)
            else:
                xx = torch.cat([x, att_map[i-1]], dim=1)
            # attention detector
            lstm_in = self.attL[i](xx)
            # LSTM Layer
#            lstm_out, self.hidden = self.lstm(lstm_in, lstm_out, self.hidden)
            # Generate attention map
#            temp = self.attM(lstm_out)
            temp = self.attM(lstm_in)
            # removal encoder
            res = self.remE[i](x, temp)
            # append to output
            att_map[i] = temp
            out[i] = res
            x = res

        # output to tensor
#        att_map = torch.FloatTensor(att_map)
#        out = torch.FloatTensor(out)

        return att_map, out

# Discriminative Network

In [ ]:
# Discriminator
class Disc(nn.Module):
    def __init__(self, batch_size=None):
        super(Disc, self).__init__()
        self.batch_size = batch_size
        self.conv0 = ConvL(3, 64, 4, 2, 1)
        self.conv1 = ConvL(64, 128, 4, 2, 1)
        self.conv2 = ConvL(128, 256, 4, 2, 1)
        self.conv3 = ConvL(256, 512, 4, 2, 1)
        self.conv4 = ConvL(512, 256, 4, 2, 1)
        self.fc = nn.Sequential(nn.Linear(256*64, 1),
                                nn.Sigmoid())

    def forward(self, inp):
      with torch.autograd.set_detect_anomaly(True):
        self.batch_size = inp.shape[0]
        x = self.conv0(inp)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = torch.flatten(x, 1)
        out = self.fc(x)

        return out

# Model Summary

In [ ]:
# Generatior Model Summary
from torchsummary import summary
summary(gen_net, (3, 256, 256), device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 256, 256]             224
       BatchNorm2d-2          [-1, 8, 256, 256]              16
         LeakyReLU-3          [-1, 8, 256, 256]               0
             ConvL-4          [-1, 8, 256, 256]               0
            Conv2d-5          [-1, 8, 256, 256]             584
       BatchNorm2d-6          [-1, 8, 256, 256]              16
         LeakyReLU-7          [-1, 8, 256, 256]               0
             ConvL-8          [-1, 8, 256, 256]               0
            Conv2d-9         [-1, 16, 256, 256]           1,168
      BatchNorm2d-10         [-1, 16, 256, 256]              32
        LeakyReLU-11         [-1, 16, 256, 256]               0
            ConvL-12         [-1, 16, 256, 256]               0
           Conv2d-13         [-1, 16, 256, 256]           2,320
      BatchNorm2d-14         [-1, 16, 2

In [ ]:
# Discriminator Model Summary
from torchsummary import summary
summary(dis_net, (3, 128, 128), device='cpu')

# Training Network

In [ ]:
# Perceptual loss by VGG16
class VGGPerceptualLoss(torch.nn.Module):
    def __init__(self, resize=True):
        super(VGGPerceptualLoss, self).__init__()
        blocks = []
        blocks.append(torchvision.models.vgg16(pretrained=True).features[:4].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[4:9].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[9:16].eval())
        blocks.append(torchvision.models.vgg16(pretrained=True).features[16:23].eval())
        for bl in blocks:
            for p in bl:
                p.requires_grad = False
        self.blocks = torch.nn.ModuleList(blocks)
        self.transform = torch.nn.functional.interpolate
        self.mean = torch.nn.Parameter(torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1))
        self.std = torch.nn.Parameter(torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1))
        self.resize = resize

    def forward(self, input, target):
        if input.shape[1] != 3:
            input = input.repeat(1, 3, 1, 1)
            target = target.repeat(1, 3, 1, 1)
        input = (input-self.mean) / self.std
        target = (target-self.mean) / self.std
        if self.resize:
            input = self.transform(input, mode='bilinear', size=(224, 224), align_corners=False)
            target = self.transform(target, mode='bilinear', size=(224, 224), align_corners=False)
        loss = 0.0
        x = input
        y = target
        for block in self.blocks:
            x = block(x)
            y = block(y)
            loss += torch.nn.functional.mse_loss(x, y)
        return loss

# Weighted Binary Cross Entropy
class WBCELoss(torch.nn.Module):
    def __init__(self, weight=True):
        super(WBCELoss, self).__init__()
        self.BCE = nn.BCEWithLogitsLoss()

    def forward(self, est, gt):
#        est_v = torch.flatten(est, start_dim=1)
#        gt_v = torch.flatten(gt, start_dim=1)

        weight = (torch.numel(gt) - torch.count_nonzero(gt)) / torch.count_nonzero(gt)
        self.BCE = nn.BCEWithLogitsLoss(pos_weight=weight)
        loss = self.BCE(est, gt)
        return loss

# Masked MSE
class MaskedMSE(torch.nn.Module):
    def __init__(self):
        super(MaskedMSE, self).__init__()
        self.MSE = nn.MSELoss(reduction='sum')

    def forward(self, est, gt, mask):
        masked_est = est * mask
        masked_gt = gt * mask

        loss = MSE(masked_est, masked_gt) / torch.count_nonzero(mask)

        return loss

In [ ]:
# learning parameters
steps = 3   # Number of progressive step
beta = 0.7  # weight for MSE step
lamb = 0.7  # weight for Semi-Supervised learning
l_rate_g = 0.0002
#l_rate_d = 0.0002

gen_net = Gen(batch_size=batch_num, step_num=steps)
#dis_net = Disc(batch_size=batch_num)

# learning loss
WBCE = WBCELoss()
MSE = nn.MSELoss()    # Mean Square Error
MMSE = MaskedMSE()
VGG = VGGPerceptualLoss(resize=True).to(device)
ADV = nn.BCELoss()    # Adversarial Loss : Binary CE

# optimizer
gen_optim = torch.optim.SGD(gen_net.parameters(), lr=l_rate_g, momentum=0.9)
#dis_optim = torch.optim.Adam(dis_net.parameters(), lr=l_rate_d)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
# Load model params
PATH = '/content/drive/My Drive/KU/4/'
SAVE_PATH = PATH + 'ARGAN2/train/'
gen_PATH = PATH + 'ARGAN2_1_gen.pth'
#dis_PATH = PATH + 'ARGAN256_dis_temp.pth'


trained = 0
if os.path.isfile(gen_PATH):
  gen_net.load_state_dict(torch.load(gen_PATH))
  trained = 212
"""
if os.path.isfile(dis_PATH):
  dis_net.load_state_dict(torch.load(dis_PATH))
  trained = 0
"""
if torch.cuda.is_available:
  print('CUDA available')
  device = "cuda:0"
gen_net.train()
#dis_net.train()

for epoch in range(49):
#    print('======================[%d epoch] running====================='
#          %(epoch+trained+1))

    # loss per epoch
    det_loss = 0.0
    rem_loss = 0.0
    adv_loss = 0.0

    for i, datas in enumerate(trainloader):

        det_err = 0.0
        rem_err = 0.0

        # from dataset
        image, matte, free = datas
        image = image.to(device)
        matte = matte.to(device)
        free = free.to(device)
        if torch.cuda.is_available:
          gen_net.cuda()

        # Generator Output
        mattes, frees = gen_net(image)

        with torch.autograd.set_detect_anomaly(True):
              """
              # train Discriminative Net
              dis_optim.zero_grad()
              # real data Error
              real_out = dis_net(free)
              real_label = torch.ones(free.shape[0],1).to(device)
              real_err = ADV(real_out, real_label)
              real_err.backward()
              # fake data Error
              fake_out = dis_net(frees[steps-1].detach())
              fake_label = torch.zeros(frees[steps-1].shape[0], 1).to(device)
              fake_err = ADV(fake_out, fake_label)
              fake_err.backward()

              dis_err = real_err + fake_err
              dis_optim.step()

#              real_loss += real_err
#              fake_loss += fake_err
              """

              # train Generative Network
              gen_optim.zero_grad()
              # for N steps
              for n in range(steps):
                  # detector loss : MSE
                  det_err += pow(beta, steps-n) * MSE(matte, mattes[n])
                  # detector loss : Weighted BCE
                  #det_err += pow(beta, steps-n) * WBCE(mattes[n], matte)
                  # removal loss : acc loss + perceptual loss
                  #rem_err += pow(beta, steps-n) * MSE(free, frees[n])
                  #r removal loss : acc loss(masked MSE)
                  rem_err += pow(beta, steps-n)* (MSE(free, frees[n])
                                                  + MMSE(frees[n], free, matte))
                  # perceptual loss VGG16
                  rem_err += VGG(free, frees[n]) / 10

              # Adversarial loss
#              out = dis_net(frees[steps-1])
              #adv_err = ADV(out, real_label)

              total_loss = det_err + rem_err
              total_loss.backward()
              gen_optim.step()

              # loss per epoch
              det_loss += det_err
              rem_loss += rem_err
              #adv_loss += adv_err


    # 1 epoch finished
    total = det_loss + rem_loss + adv_loss
    print('[%d epoch]\t det : %f, rem : %f, adv : %f, total : %f'
            %(epoch+trained+1, det_loss, rem_loss, adv_loss, total))
#    print('\t\t total loss = %f' %(det_loss + rem_loss + adv_loss))
    torch.save(gen_net.state_dict(), gen_PATH)
#    torch.save(dis_net.state_dict(), dis_PATH)

    img_fname = SAVE_PATH + str(epoch+trained+1) + "_img.jpg"
    mat_fname = SAVE_PATH + str(epoch+trained+1) + "_matt.jpg"
    fre_fname = SAVE_PATH + str(epoch+trained+1) + "_free.jpg"

    # data out : to 'cpu'
    img_out = image.cpu()
    save_batch(img_out, dprow, img_fname)
    matt_out = mattes[steps-1].cpu()
    save_batch(matt_out, dprow, mat_fname)
    free_out = frees[steps-1].cpu()
    save_batch(free_out, dprow, fre_fname)


CUDA available
[213 epoch]	 det : 18.797579, rem : 127.871910, adv : 0.000000, total : 146.669495
[214 epoch]	 det : 18.193010, rem : 127.009743, adv : 0.000000, total : 145.202759
[215 epoch]	 det : 19.148127, rem : 128.450500, adv : 0.000000, total : 147.598633
[216 epoch]	 det : 18.208004, rem : 126.937866, adv : 0.000000, total : 145.145874
[217 epoch]	 det : 18.195364, rem : 126.784698, adv : 0.000000, total : 144.980057
[218 epoch]	 det : 17.705355, rem : 125.869087, adv : 0.000000, total : 143.574448
[219 epoch]	 det : 17.900108, rem : 125.364899, adv : 0.000000, total : 143.265015
[220 epoch]	 det : 17.673428, rem : 125.418114, adv : 0.000000, total : 143.091537
[221 epoch]	 det : 17.576696, rem : 124.670067, adv : 0.000000, total : 142.246765
[222 epoch]	 det : 16.832941, rem : 123.903053, adv : 0.000000, total : 140.735992
[223 epoch]	 det : 17.051580, rem : 123.528633, adv : 0.000000, total : 140.580215
[224 epoch]	 det : 17.370462, rem : 123.979324, adv : 0.000000, total : 

# Test Network

In [ ]:
# Model parameters
steps = 3   # Number of progressive step

gen_net = Gen(batch_size=batch_num, step_num=steps)
#dis_net = Disc(batch_size=batch_num)

Metrics

In [ ]:
# Balanced Error Rate
class BERScore(nn.Module):
  def __init__(self, thresh=None):
    super().__init__()
    self.thr = thresh

  def forward(self, est, gt):
    est_b = (est>self.thr).float()

    conf_mat = est_b / gt

    tp = torch.sum(conf_mat == 1).item()
    fp = torch.sum(conf_mat == float('inf')).item()
    tn = torch.sum(torch.isnan(conf_mat)).item()
    fn = torch.sum(conf_mat == 0).item()

    sensit = tp / (tp+fn)
    specif = tn / (tn+fp)
    return (1 - (sensit + specif)/2 )

# Balanced Error Rate & sensitivity & specificity
class BERScores(nn.Module):
  def __init__(self, thresh=None):
    super().__init__()
    self.thr = thresh

  def forward(self, est, gt):
    est_b = (est>self.thr).float()

    conf_mat = est_b / gt

    tp = torch.sum(conf_mat == 1).item()
    fp = torch.sum(conf_mat == float('inf')).item()
    tn = torch.sum(torch.isnan(conf_mat)).item()
    fn = torch.sum(conf_mat == 0).item()

    sensit = tp / (tp+fn)
    specif = tn / (tn+fp)
    return sensit, specif, (1 - (sensit + specif)/2 )

# Root Mean Square
class RMSEScore(nn.Module):
  def __init__(self):
    super().__init__()
    self.mse = nn.MSELoss()

  def forward(self, est, gt):
    score = torch.sqrt(self.mse(est, gt))
    return float(score)

BER = BERScore(thresh=0.5)
BERs = BERScores(thresh=0.5)
RMSE = RMSEScore()

In [ ]:
# image transforms
def rgb2lab(image):
  temp = image.cpu()
  out = torch.empty_like(temp)
  temp = np.transpose(temp, (0,2,3,1))
  for i in range(temp.shape[0]):
    out[i] = torch.FloatTensor(np.transpose(color.rgb2lab(np.array(temp[i])),
                                          (2,0,1)))
  out = out.to(device)
  return out

matt2src = transforms.Compose([
                               transforms.Resize(size=(480,640)),
])
free2src = transforms.Compose([
                               transforms.Resize(size=(480,640)),
                               transforms.Lambda(rgb2lab)
])
free2lab = transforms.Compose([
                               transforms.Lambda(rgb2lab)
])
gray2rgb = transforms.Compose([
#                               transforms.Grayscale(num_output_channels=3),
                               transforms.Lambda(lambda x: x.repeat(1,3,1,1))
])
grays2rgb = transforms.Compose([
                                transforms.Lambda(lambda x: x.repeat(1,1,3,1,1))
])

def save_total_batch(images, nrow, PATH):
  img = torch.zeros_like(images[0])
  img = images[0]
  for i in range(1, images.shape[0]):
    img = torch.cat((img, images[i]))
  img_out = torchvision.utils.make_grid(img, nrow=nrow)
  img_out = np.transpose(img_out.detach().numpy().astype('float64'), (1, 2, 0))
  img_out = (255*img_out).astype('uint8')
  img_out = Image.fromarray(img_out)
  img_out.save(PATH)

Network Evaluation with batch_size 1

In [ ]:
batch_num = 1
dprow = 1

test_img = ARGAN_Dataset(test_path, src_trans=img2tensor, matt_trans=matt2tensor, is_test=True)
testloader = torch.utils.data.DataLoader(test_img, batch_size=1, shuffle=False)

In [ ]:
# Evaluation with size recovering
if torch.cuda.is_available:
  gen_net.cuda()
  device = "cuda:0"

gen_net.eval()
for m in gen_net.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.track_running_stats = False

PATH = '/content/drive/My Drive/KU/4/'
gen_PATH = PATH + 'ARGAN2_1_gen.pth'
SAVE_PATH = PATH + 'ARGAN2/test2/'

if os.path.isfile(gen_PATH):
  gen_net.load_state_dict(torch.load(gen_PATH))
else:
  print("No trained Gen Net")

total_BER = []
total_RMSE = []

for i, datas in enumerate(testloader):
  with torch.no_grad():
    image, matte, free, src_matte, src_free = datas
    image = image.to(device)
    matte = matte.to(device)
    free = free.to(device)
    src_matte = src_matte.to(device)
    src_free = src_free.to(device)

    mattes, frees = gen_net(image)

    image_ = matt2src(image)
    mattes_ = matt2src(mattes[steps-1])
    frees_ = free2src(frees[steps-1])
    src_free_lab = free2src(src_free)

    ber = BER(mattes_, src_matte)
    rmse = RMSE(frees_, src_free_lab)
    print("[%d image]\tber : %f, rmse : %f" %(i+1, ber, rmse))

    total_BER.append(ber)
    total_RMSE.append(rmse)


    img_fname = SAVE_PATH + str(i) + "_img.jpg"
    mat_fname = SAVE_PATH + str(i) + "_matt.jpg"
    fre_fname = SAVE_PATH + str(i) + "_free.jpg"
    gt_m_fname = SAVE_PATH + str(i) + "_matt_gt.jpg"
    gt_f_fname = SAVE_PATH + str(i) + "_free_gt.jpg"

    # data out : to 'cpu'
    img_out = image_.cpu()
    save_batch(img_out, dprow, img_fname)
    matt_out = mattes_.cpu()
    save_batch(matt_out, dprow, mat_fname)
    free_out = frees_.cpu()
    save_batch_LAB(free_out, dprow, fre_fname)
    gt_m_out = src_matte.cpu()
    save_batch(gt_m_out, dprow, gt_m_fname)
    gt_f_out = src_free.cpu()
    save_batch(gt_f_out, dprow, gt_f_fname)

    torch.cuda.empty_cache()

avg_BER = np.mean(total_BER)
avg_RMSE = np.mean(total_RMSE)

print("DONE============")
print("BER : %f" %(avg_BER))
print("RMSE : %f" %(avg_RMSE))

[1 image]	ber : 0.001999, rmse : 3.720757
[2 image]	ber : 0.026759, rmse : 6.783995
[3 image]	ber : 0.035629, rmse : 4.561716
[4 image]	ber : 0.003823, rmse : 3.150646
[5 image]	ber : 0.006662, rmse : 2.285603
[6 image]	ber : 0.032405, rmse : 2.901250
[7 image]	ber : 0.060133, rmse : 2.654407
[8 image]	ber : 0.005121, rmse : 2.282096
[9 image]	ber : 0.002138, rmse : 4.458961
[10 image]	ber : 0.005601, rmse : 4.717868
[11 image]	ber : 0.030401, rmse : 9.506086
[12 image]	ber : 0.034204, rmse : 4.328887
[13 image]	ber : 0.036287, rmse : 4.222516
[14 image]	ber : 0.035167, rmse : 4.263539
[15 image]	ber : 0.035884, rmse : 4.306076
[16 image]	ber : 0.027839, rmse : 5.750278
[17 image]	ber : 0.027617, rmse : 5.542061
[18 image]	ber : 0.035592, rmse : 4.323061
[19 image]	ber : 0.036417, rmse : 4.459897
[20 image]	ber : 0.036922, rmse : 4.414705
[21 image]	ber : 0.025153, rmse : 4.130602
[22 image]	ber : 0.030415, rmse : 2.863933
[23 image]	ber : 0.024772, rmse : 3.125119
[24 image]	ber : 0.0

In [ ]:
# Evaluation with size recovering
# No images saved
if torch.cuda.is_available:
  gen_net.cuda()
  device = "cuda:0"

gen_net.eval()
for m in gen_net.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.track_running_stats = False

PATH = '/content/drive/My Drive/KU/4/'
#gen_PATH = PATH + 'ARGAN256_gen_BL.pth'
gen_PATH = PATH + 'ARGAN2_1_gen.pth'

if os.path.isfile(gen_PATH):
  gen_net.load_state_dict(torch.load(gen_PATH))
else:
  print("No trained Gen Net")

total_BER = []
total_sens = []
total_spec = []
total_RMSE = []

for i, datas in enumerate(testloader):
  with torch.no_grad():
    image, matte, free, src_matte, src_free = datas
    image = image.to(device)
    matte = matte.to(device)
    free = free.to(device)
    src_matte = src_matte.to(device)
    src_free = src_free.to(device)

    mattes, frees = gen_net(image)

    image_ = matt2src(image)
    mattes_ = matt2src(mattes[steps-1])
    frees_ = free2src(frees[steps-1])
    src_free_lab = free2src(src_free)

    sens, spec, ber = BERs(mattes_, src_matte)
    rmse = RMSE(frees_, src_free_lab)
    print("[%d image]\tber : %f (%f / %f), rmse : %f" %(i+1, ber, sens, spec, rmse))

    total_BER.append(ber)
    total_sens.append(sens)
    total_spec.append(spec)
    total_RMSE.append(rmse)

    torch.cuda.empty_cache()

avg_BER = np.mean(total_BER)
avg_sens = np.mean(total_sens)
avg_spec = np.mean(total_spec)
avg_RMSE = np.mean(total_RMSE)

print("DONE============")
print("BER : %f" %(avg_BER))
print("sens : %f" %(avg_sens))
print("spec : %f" %(avg_spec))
print("RMSE : %f" %(avg_RMSE))

[1 image]	ber : 0.001999 (0.999231 / 0.996771), rmse : 3.720757
[2 image]	ber : 0.026759 (0.964790 / 0.981693), rmse : 6.783995
[3 image]	ber : 0.035629 (0.935746 / 0.992997), rmse : 4.561716
[4 image]	ber : 0.003823 (0.996894 / 0.995460), rmse : 3.150646
[5 image]	ber : 0.006662 (0.987252 / 0.999424), rmse : 2.285603
[6 image]	ber : 0.032405 (0.939335 / 0.995855), rmse : 2.901250
[7 image]	ber : 0.060133 (0.883570 / 0.996164), rmse : 2.654407
[8 image]	ber : 0.005121 (0.992158 / 0.997601), rmse : 2.282096
[9 image]	ber : 0.002138 (0.997421 / 0.998303), rmse : 4.458961
[10 image]	ber : 0.005601 (0.999869 / 0.988929), rmse : 4.717868
[11 image]	ber : 0.030401 (0.949230 / 0.989967), rmse : 9.506086
[12 image]	ber : 0.034204 (0.988929 / 0.942662), rmse : 4.328887
[13 image]	ber : 0.036287 (0.993980 / 0.933446), rmse : 4.222516
[14 image]	ber : 0.035167 (0.996741 / 0.932925), rmse : 4.263539
[15 image]	ber : 0.035884 (0.996047 / 0.932186), rmse : 4.306075
[16 image]	ber : 0.027839 (0.99831

In [ ]:
# Evaluation without resizing
if torch.cuda.is_available:
  gen_net.cuda()
  device = "cuda:0"

gen_net.eval()
for m in gen_net.modules():
    if isinstance(m, nn.BatchNorm2d):
        m.track_running_stats = False

PATH = '/content/drive/My Drive/KU/4/'
gen_PATH = PATH + 'ARGAN2_1_gen.pth'

if os.path.isfile(gen_PATH):
  gen_net.load_state_dict(torch.load(gen_PATH))
else:
  print("No trained Gen Net")

total_BER = []
total_sens = []
total_spec = []
total_RMSE = []

for i, datas in enumerate(testloader):
  with torch.no_grad():
    image, matte, free, src_matte, src_free = datas
    image = image.to(device)
    matte = matte.to(device)
    free = free.to(device)

    mattes, frees = gen_net(image)

    frees_lab = free2lab(frees[steps-1])
    free_lab = free2lab(free)

    sens, spec, ber = BERs(mattes[steps-1], matte)
    rmse = RMSE(frees_lab, free_lab)
    print("[%d image]\tber : %f (%f / %f), rmse : %f" %(i+1, ber,sens, spec, rmse))

    total_BER.append(ber)
    total_sens.append(sens)
    total_spec.append(spec)
    total_RMSE.append(rmse)

    torch.cuda.empty_cache()

avg_BER = np.mean(total_BER)
avg_sens = np.mean(total_sens)
avg_spec = np.mean(total_spec)
avg_RMSE = np.mean(total_RMSE)

print("DONE============")
print("BER : %f" %(avg_BER))
print("Sens : %f" %(avg_sens))
print("Spec : %f" %(avg_spec))
print("RMSE : %f" %(avg_RMSE))

# Output Images

In [ ]:
batch_num = 1
dprow = 1

test_img = ARGAN_Dataset(test_path, src_trans=img2tensor, matt_trans=matt2tensor, is_test=True)
testloader = torch.utils.data.DataLoader(test_img, batch_size=1, shuffle=False)

def save_recur_batch(src_img, images, gt_img, nrow, PATH):
  img = torch.zeros_like(images[0])
  img = src_img
  for i in range(0, images.shape[0]):
    img = torch.cat((img, images[i]))
  img = torch.cat((img, gt_img))
  img_out = torchvision.utils.make_grid(img, nrow=nrow)
  img_out = np.transpose(img_out.detach().numpy().astype('float64'), (1, 2, 0))
  img_out = (255*img_out).astype('uint8')
  img_out = Image.fromarray(img_out)
  img_out.save(PATH)

In [ ]:
#### as 1 image
if torch.cuda.is_available:
  gen_net.cuda()
  device = "cuda:0"

gen_net.eval()

PATH = '/content/drive/My Drive/KU/4/'
gen_PATH = PATH + 'ARGAN2_1_gen.pth'
SAVE_PATH = PATH + 'ARGAN2/Final_grad2/'

if os.path.isfile(gen_PATH):
  gen_net.load_state_dict(torch.load(gen_PATH))
else:
  print("No trained Gen Net")

for i, datas in enumerate(testloader):
  with torch.no_grad():
    image, matte, free, src_matte, src_free = datas
    image = image.to(device)
    matte = matte.to(device)
    free = free.to(device)
    src_matte = src_matte.to(device)
    src_free = src_free.to(device)

    mattes, frees = gen_net(image)

    image_ = matt2src(image)
    mattes_ = matt2src(mattes[steps-1])
    mattes_ = gray2rgb(mattes_)
    frees_ = matt2src(frees[steps-1])

    fin_fname = SAVE_PATH + str(i) + "_fin.jpg"
    matt_fname = SAVE_PATH + str(i) + "_mattes.jpg"
    fre_fname = SAVE_PATH + str(i) + "_frees.jpg"

    """
    # Outputs & GT
    img_out = image_.cpu()
    matt_out = mattes_.cpu()
    free_out = frees_.cpu()
    gt_m_out = src_matte.cpu()
    gt_f_out = src_free.cpu()
    fin_out = torch.cat((img_out, matt_out, gt_m_out, free_out, gt_f_out))
    save_batch(fin_out, 3, fin_fname)
    """

    # gradual outputs
    image = image.cpu()
    matte = matte.cpu()
    matte = gray2rgb(matte)
    mattes = grays2rgb(mattes)
    mattes_out = mattes.cpu()

    free = free.cpu()
    frees_out = frees.cpu()

    save_recur_batch(image, mattes_out, matte, 5, matt_fname)
    save_recur_batch(image, frees_out, free, 5, fre_fname)

    torch.cuda.empty_cache()

print("DONE============")

DONE============
